In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
n_LeadTimeSegments = 5
n_AtHomeDCs = 8

# Penalty Cost Matrix ($)
ILLWILLcost_matrix = np.array([[10, 10, 10, 10, 10, 10, 10, 10],
                               [200, 20, 20, 20, 20, 20, 20, 20],
                               [300, 30, 30, 30, 30, 30, 30, 30],
                               [400, 40, 40, 40, 40, 40, 40, 40],
                               [500, 50, 50, 50, 50, 50, 50, 50]])

#Non_ve_LeadTimeSegments = np.array([0, 0, 0, 0, 0])
#Non_ve_AtHomeDCs = np.array([0, 0, 0, 0, 0, 0, 0, 0])

# Demand Matrix (in number of Pallets)
Pallet_demands = np.array([243, 178, 119, 276, 340, 98, 153, 310])


# Logistics Cost Matrix ($)
LTL_Spot_Fixing_Cost = np.array([6000, 4500, 4500, 4500, 4500])
FTL_Spot_Fixing_Cost = np.array([4000, 3000, 3000, 3000, 3000])

# Transport Capacity (in number of Pallets)
Low_Truck_Capacity = np.array([10, 10, 10, 10, 10])
High_Truck_Capacity = np.array([20, 20, 20, 20, 20])


In [3]:
#model = LpProblem("FTL-Maximization-Problem", LpMinimize)

In [4]:
variable_names_shipment = [str(i)+str(j) for j in range(1, n_AtHomeDCs+1) for i in range(1, n_LeadTimeSegments+1)]
variable_names_shipment.sort()
print("Variable Indices:", variable_names_shipment)

Variable Indices: ['11', '12', '13', '14', '15', '16', '17', '18', '21', '22', '23', '24', '25', '26', '27', '28', '31', '32', '33', '34', '35', '36', '37', '38', '41', '42', '43', '44', '45', '46', '47', '48', '51', '52', '53', '54', '55', '56', '57', '58']


In [5]:
variable_names_LTL_Status = [str(i) for i in range(1, n_LeadTimeSegments+1)]
variable_names_LTL_Status.sort()
print("Variable Indices:", variable_names_LTL_Status)

Variable Indices: ['1', '2', '3', '4', '5']


In [6]:
variable_names_FTL_Status = [str(i) for i in range(1, n_LeadTimeSegments+1)]
variable_names_LTL_Status.sort()
print("Variable Indices:", variable_names_FTL_Status)

Variable Indices: ['1', '2', '3', '4', '5']


In [7]:
DV_shipments = LpVariable.matrix("X", variable_names_shipment, cat = "Integer", lowBound= 0 )
allocation = np.array(DV_shipments).reshape(5,8)
print("Decision Variable/Allocation Matrix: ")
print(allocation)

Decision Variable/Allocation Matrix: 
[[X_11 X_12 X_13 X_14 X_15 X_16 X_17 X_18]
 [X_21 X_22 X_23 X_24 X_25 X_26 X_27 X_28]
 [X_31 X_32 X_33 X_34 X_35 X_36 X_37 X_38]
 [X_41 X_42 X_43 X_44 X_45 X_46 X_47 X_48]
 [X_51 X_52 X_53 X_54 X_55 X_56 X_57 X_58]]


In [8]:
DV_status_LTL = LpVariable.matrix("Y", variable_names_LTL_Status, cat = "Integer", lowBound= 0 )
status_LTL = np.array(DV_status_LTL).reshape(5,1)
print("Decision Variable/LTL Status Matrix: ")
print(status_LTL)

Decision Variable/LTL Status Matrix: 
[[Y_1]
 [Y_2]
 [Y_3]
 [Y_4]
 [Y_5]]


In [9]:
DV_status_FTL = LpVariable.matrix("Z", variable_names_FTL_Status, cat = "Integer", lowBound= 0 )
status_FTL = np.array(DV_status_FTL).reshape(5,1)
print("Decision Variable/FTL Status Matrix: ")
print(status_FTL)

Decision Variable/FTL Status Matrix: 
[[Z_1]
 [Z_2]
 [Z_3]
 [Z_4]
 [Z_5]]


In [10]:
variable_names_FTL_Indicator = [str(j) for j in range(1, n_AtHomeDCs+1)]
variable_names_FTL_Indicator.sort()
print("Variable Indices:", variable_names_FTL_Indicator)

Variable Indices: ['1', '2', '3', '4', '5', '6', '7', '8']


In [11]:
DV_Indicator_FTL = LpVariable.matrix("W", variable_names_FTL_Indicator, cat = "Integer", lowBound= 0 )
Indicator_FTL = np.array(DV_Indicator_FTL).reshape(1,8)
print("Decision Variable/FTL Indicator Matrix: ")
print(Indicator_FTL)

Decision Variable/FTL Indicator Matrix: 
[[W_1 W_2 W_3 W_4 W_5 W_6 W_7 W_8]]


In [12]:
#obj_func1 = lpSum(allocation*ILLWILLcost_matrix) 
#obj_func2 = (LTL_Spot_Fixing_Cost*status_LTL)
#obj_func3 = (FTL_Spot_Fixing_Cost*status_FTL)
#obj_func = obj_func1 + obj_func2 + obj_func3
#print(obj_func)
#model +=  obj_func1 + obj_func2 + obj_func3
#print(model)

In [13]:
import operator
def dotproduct(vec1, vec2):
    return sum(map(operator.mul, vec1, vec2))

In [14]:
a = [LTL_Spot_Fixing_Cost] 
b = [status_LTL]
o = [[dotproduct(m, n)] for m, n in zip(a, b)]
c = [FTL_Spot_Fixing_Cost] 
d = [status_FTL]
p = [[dotproduct(m, n)] for m, n in zip(c, d)]
print(o)
print(p)

[[array([6000*Y_1 + 4500*Y_2 + 4500*Y_3 + 4500*Y_4 + 4500*Y_5 + 0],
      dtype=object)]]
[[array([4000*Z_1 + 3000*Z_2 + 3000*Z_3 + 3000*Z_4 + 3000*Z_5 + 0],
      dtype=object)]]


In [15]:
obj_func = lpSum(allocation*ILLWILLcost_matrix) + o + p + Indicator_FTL*20
print(obj_func)
#model +=  obj_func
#print(model)

20*W_1 + 20*W_2 + 20*W_3 + 20*W_4 + 20*W_5 + 20*W_6 + 20*W_7 + 20*W_8 + 10*X_11 + 10*X_12 + 10*X_13 + 10*X_14 + 10*X_15 + 10*X_16 + 10*X_17 + 10*X_18 + 200*X_21 + 20*X_22 + 20*X_23 + 20*X_24 + 20*X_25 + 20*X_26 + 20*X_27 + 20*X_28 + 300*X_31 + 30*X_32 + 30*X_33 + 30*X_34 + 30*X_35 + 30*X_36 + 30*X_37 + 30*X_38 + 400*X_41 + 40*X_42 + 40*X_43 + 40*X_44 + 40*X_45 + 40*X_46 + 40*X_47 + 40*X_48 + 500*X_51 + 50*X_52 + 50*X_53 + 50*X_54 + 50*X_55 + 50*X_56 + 50*X_57 + 50*X_58 + 6000*Y_1 + 4500*Y_2 + 4500*Y_3 + 4500*Y_4 + 4500*Y_5 + 4000*Z_1 + 3000*Z_2 + 3000*Z_3 + 3000*Z_4 + 3000*Z_5


In [16]:
e = [Low_Truck_Capacity] 
b = [status_LTL]
q = [[dotproduct(m, n)] for m, n in zip(e, b)]
f = [High_Truck_Capacity] 
d = [status_FTL]
r = [[dotproduct(m, n)] for m, n in zip(f, d)]
print(q)
print(r)

[[array([10*Y_1 + 10*Y_2 + 10*Y_3 + 10*Y_4 + 10*Y_5 + 0], dtype=object)]]
[[array([20*Z_1 + 20*Z_2 + 20*Z_3 + 20*Z_4 + 20*Z_5 + 0], dtype=object)]]


In [17]:
model = LpProblem("FTL-Maximization-Problem", LpMinimize)
model +=  obj_func

In [18]:
for i in range(n_LeadTimeSegments):
    #print(-lpSum(allocation[i][j] for j in range(n_AtHomeDCs)) + (status_FTL[i] * High_Truck_Capacity[i]) + (status_LTL[i] * Low_Truck_Capacity[i]) >= Non_ve_LeadTimeSegments[i])
    print(-lpSum(allocation[i][j] for j in range(n_AtHomeDCs)) + (status_FTL[i] * High_Truck_Capacity[i]) + (status_LTL[i] * Low_Truck_Capacity[i]) >= 0)
    model += (-lpSum(allocation[i][j] for j in range(n_AtHomeDCs)) + (status_FTL[i] * High_Truck_Capacity[i]) + (status_LTL[i] * Low_Truck_Capacity[i])) >= 0, "Lot-Sizing Constraints " + str(i)
    
for j in range(n_AtHomeDCs):
    #print(- lpSum(allocation[i][j] for i in range(n_LeadTimeSegments)) + Pallet_demands[j] == Non_ve_AtHomeDCs[j])
    print(-lpSum(allocation[i][j] for i in range(n_LeadTimeSegments)) + Pallet_demands[j] == 0)
    model += (-lpSum(allocation[i][j] for i in range(n_LeadTimeSegments)) + Pallet_demands[j]) == 0, "Replenishment Constraints " + str(j)

for j in range(n_AtHomeDCs):
    #print(- lpSum(allocation[i][j] for i in range(n_LeadTimeSegments)) + Indicator_FTL*20 >= Non_ve_AtHomeDCs[j])
    print(-lpSum(allocation[i][j] for i in range(n_LeadTimeSegments)) + (DV_Indicator_FTL[j]*20) >= 0)
    model += (-lpSum(allocation[i][j] for i in range(n_LeadTimeSegments)) + (DV_Indicator_FTL[j]*20)) >= 0, "Truck Loading Constraints " + str(j)

-X_11 - X_12 - X_13 - X_14 - X_15 - X_16 - X_17 - X_18 + 10*Y_1 + 20*Z_1 >= 0
-X_21 - X_22 - X_23 - X_24 - X_25 - X_26 - X_27 - X_28 + 10*Y_2 + 20*Z_2 >= 0
-X_31 - X_32 - X_33 - X_34 - X_35 - X_36 - X_37 - X_38 + 10*Y_3 + 20*Z_3 >= 0
-X_41 - X_42 - X_43 - X_44 - X_45 - X_46 - X_47 - X_48 + 10*Y_4 + 20*Z_4 >= 0
-X_51 - X_52 - X_53 - X_54 - X_55 - X_56 - X_57 - X_58 + 10*Y_5 + 20*Z_5 >= 0
-X_11 - X_21 - X_31 - X_41 - X_51 = -243
-X_12 - X_22 - X_32 - X_42 - X_52 = -178
-X_13 - X_23 - X_33 - X_43 - X_53 = -119
-X_14 - X_24 - X_34 - X_44 - X_54 = -276
-X_15 - X_25 - X_35 - X_45 - X_55 = -340
-X_16 - X_26 - X_36 - X_46 - X_56 = -98
-X_17 - X_27 - X_37 - X_47 - X_57 = -153
-X_18 - X_28 - X_38 - X_48 - X_58 = -310
20*W_1 - X_11 - X_21 - X_31 - X_41 - X_51 >= 0
20*W_2 - X_12 - X_22 - X_32 - X_42 - X_52 >= 0
20*W_3 - X_13 - X_23 - X_33 - X_43 - X_53 >= 0
20*W_4 - X_14 - X_24 - X_34 - X_44 - X_54 >= 0
20*W_5 - X_15 - X_25 - X_35 - X_45 - X_55 >= 0
20*W_6 - X_16 - X_26 - X_36 - X_46 - X_56 >= 0
2

In [19]:
print(model)

FTL-Maximization-Problem:
MINIMIZE
20*W_1 + 20*W_2 + 20*W_3 + 20*W_4 + 20*W_5 + 20*W_6 + 20*W_7 + 20*W_8 + 10*X_11 + 10*X_12 + 10*X_13 + 10*X_14 + 10*X_15 + 10*X_16 + 10*X_17 + 10*X_18 + 200*X_21 + 20*X_22 + 20*X_23 + 20*X_24 + 20*X_25 + 20*X_26 + 20*X_27 + 20*X_28 + 300*X_31 + 30*X_32 + 30*X_33 + 30*X_34 + 30*X_35 + 30*X_36 + 30*X_37 + 30*X_38 + 400*X_41 + 40*X_42 + 40*X_43 + 40*X_44 + 40*X_45 + 40*X_46 + 40*X_47 + 40*X_48 + 500*X_51 + 50*X_52 + 50*X_53 + 50*X_54 + 50*X_55 + 50*X_56 + 50*X_57 + 50*X_58 + 6000*Y_1 + 4500*Y_2 + 4500*Y_3 + 4500*Y_4 + 4500*Y_5 + 4000*Z_1 + 3000*Z_2 + 3000*Z_3 + 3000*Z_4 + 3000*Z_5 + 0
SUBJECT TO
Lot_Sizing_Constraints_0: - X_11 - X_12 - X_13 - X_14 - X_15 - X_16 - X_17
 - X_18 + 10 Y_1 + 20 Z_1 >= 0

Lot_Sizing_Constraints_1: - X_21 - X_22 - X_23 - X_24 - X_25 - X_26 - X_27
 - X_28 + 10 Y_2 + 20 Z_2 >= 0

Lot_Sizing_Constraints_2: - X_31 - X_32 - X_33 - X_34 - X_35 - X_36 - X_37
 - X_38 + 10 Y_3 + 20 Z_3 >= 0

Lot_Sizing_Constraints_3: - X_41 - X_42 - X_4

In [20]:
model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)

Optimal


In [21]:
print("Total Logistics Cost:", model.objective.value())

# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Logistics Cost: 304240.0
W_1 = 13.0
W_2 = 9.0
W_3 = 6.0
W_4 = 14.0
W_5 = 17.0
W_6 = 5.0
W_7 = 8.0
W_8 = 16.0
X_11 = 240.0
X_12 = 0.0
X_13 = 0.0
X_14 = 0.0
X_15 = 0.0
X_16 = 0.0
X_17 = 0.0
X_18 = 0.0
X_21 = 3.0
X_22 = 178.0
X_23 = 119.0
X_24 = 276.0
X_25 = 340.0
X_26 = 98.0
X_27 = 153.0
X_28 = 310.0
X_31 = 0.0
X_32 = 0.0
X_33 = 0.0
X_34 = 0.0
X_35 = 0.0
X_36 = 0.0
X_37 = 0.0
X_38 = 0.0
X_41 = 0.0
X_42 = 0.0
X_43 = 0.0
X_44 = 0.0
X_45 = 0.0
X_46 = 0.0
X_47 = 0.0
X_48 = 0.0
X_51 = 0.0
X_52 = 0.0
X_53 = 0.0
X_54 = 0.0
X_55 = 0.0
X_56 = 0.0
X_57 = 0.0
X_58 = 0.0
Y_1 = 0.0
Y_2 = 0.0
Y_3 = 0.0
Y_4 = 0.0
Y_5 = 0.0
Z_1 = 12.0
Z_2 = 74.0
Z_3 = 0.0
Z_4 = 0.0
Z_5 = 0.0


In [22]:
for j in range(n_AtHomeDCs):
    print("No. FTLs for the At-Home DC", str(j+1))
    print(DV_Indicator_FTL[j].value())

No. FTLs for the At-Home DC 1
13.0
No. FTLs for the At-Home DC 2
9.0
No. FTLs for the At-Home DC 3
6.0
No. FTLs for the At-Home DC 4
14.0
No. FTLs for the At-Home DC 5
17.0
No. FTLs for the At-Home DC 6
5.0
No. FTLs for the At-Home DC 7
8.0
No. FTLs for the At-Home DC 8
16.0


In [59]:
import pandas as pd
import csv

In [76]:
for v in model.variables():
    try:
        #print(v.name,"=", v.value())
        #data = [[v.name],[v.value]]
        dict = {'Decision Variables Names': v.name, 'Decision Variables Velues': v.value()}
        print(dict)
        #file = open('Output.csv', 'w+', newline ='')
        #with file:    
            #write = csv.writer(file)
            #write.writerows(data)
        df = pd.DataFrame(dict)
        df.to_csv = ('OP.csv')
    except:
        print("--------------------------")

{'Decision Variables Names': 'W_1', 'Decision Variables Velues': 13.0}
--------------------------
{'Decision Variables Names': 'W_2', 'Decision Variables Velues': 9.0}
--------------------------
{'Decision Variables Names': 'W_3', 'Decision Variables Velues': 6.0}
--------------------------
{'Decision Variables Names': 'W_4', 'Decision Variables Velues': 14.0}
--------------------------
{'Decision Variables Names': 'W_5', 'Decision Variables Velues': 17.0}
--------------------------
{'Decision Variables Names': 'W_6', 'Decision Variables Velues': 5.0}
--------------------------
{'Decision Variables Names': 'W_7', 'Decision Variables Velues': 8.0}
--------------------------
{'Decision Variables Names': 'W_8', 'Decision Variables Velues': 16.0}
--------------------------
{'Decision Variables Names': 'X_11', 'Decision Variables Velues': 240.0}
--------------------------
{'Decision Variables Names': 'X_12', 'Decision Variables Velues': 0.0}
--------------------------
{'Decision Variables N